# 5.2 インストラクションチューニング

## 5.2.1 インストラクションチューニングの基礎

In [ ]:
data = [{ "from": "human", "value": "冷たいシャワーを毎日浴びることの長所と短所について、短いパラグラフを書いてください。" }, { "from": "gpt", "value": "冷たいシャワーを毎日浴びることのメリットは、注意力を高め、気分を高揚させること、健康全般を改善すること、ガスや電気を使ってお湯を沸かさないので光熱費の節約になることです。一方、冷たいシャワーはあまり歓迎されるものではありません。特に寒い冬には、冷たいシャワーに浸かるという精神的な障壁を超えられず、失敗や失望を考えてしまうかもしれません。" }]

prompt_template_example = "{from}: {value}"
instruction_text = ""
for d in data[:-1]:
    instruction_text += prompt_template_example.format(**d)
response_text = prompt_template_example.format(**data[-1])

print("指示部分:")
print(instruction_text)
print("\n応答部分:")
print(response_text)

## 5.2.2 インストラクションチューニングの実装

In [ ]:
from datasets import load_dataset

ds = load_dataset("databricks/databricks-dolly-15k")
print(ds["train"]) 
# Dataset({
#     features: ['instruction', 'context', 'response', 'category'],
#     num_rows: 15011
# })

In [ ]:
ds_train = ds["train"].filter(lambda x: x["context"] == "")
print(f"コンテキスト空のデータ: {ds_train.num_rows}") # 10544

In [ ]:
prompt_template = """\
### Question: {instruction}
### Answer: {response}{eos_token}"""

def format_input(example):
    """バッチ処理用のフォーマット関数"""
    texts = []
    for instruction, response in zip(example['instruction'], example['response']):
        text = prompt_template.format(
            instruction=instruction,
            response=response,
            eos_token=tokenizer.eos_token
        )
        texts.append(text)
    return texts

sample = ds_train[0]
print("サンプルデータ:")
print(f"  instruction: {sample['instruction'][:50]}...")
print(f"  response: {sample['response'][:50]}...")

#  サンプルデータ:
#  instruction: Which is a species of fish? Tope or Rope...
#  response: Tope...

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto" if torch.cuda.is_available() else "cpu",
)

print(f"パラメータ数: {sum(p.numel() for p in model.parameters()):,}") # パラメータ数: 124,439,808

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib

fig, ax = plt.subplots()

lengths = [len(tokenizer.encode(text)) for text in format_input(ds_train)]
ax.hist(lengths, bins=200)
ax.set_xlim(0, 1000)
ax.set_xlabel("トークン数")
ax.set_ylabel("レコード数")
fig.savefig("./output/histogram.png", dpi=300, bbox_inches="tight")

In [ ]:
max_length = 512

def token_length_filter(x):
    text = prompt_template.format(
        instruction=x["instruction"],
        response=x["response"],
        eos_token=tokenizer.eos_token
    )
    return len(tokenizer.encode(text)) <= max_length

ds_train = ds_train.filter(token_length_filter)
print(f"トークン数フィルタ後: {ds_train.num_rows}") # トークン数フィルタ後: 10400

In [ ]:
prompt_template_infer = """\
### Question: {instruction}
### Answer: """
response_template = "### Answer:"

@torch.inference_mode()
def inference(model, tokenizer, user_input):
    prompt = prompt_template_infer.format(instruction=user_input)
    device = next(model.parameters()).device
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    output = model.generate(
        input_ids,
        max_new_tokens=128,
        do_sample=False,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=False)
    response_start = generated_text.find(response_template) + len(response_template)
    response_end = generated_text.find(tokenizer.eos_token, response_start)
    if response_end == -1:
        response_end = len(generated_text)
    response = generated_text[response_start:response_end].strip()
    return response

In [ ]:
test_questions = [
    "What is the capital of Japan?"
]

print("チューニング前の応答:")
print("="*80)
before_responses = {}
for question in test_questions:
    response = inference(model, tokenizer, question)
    before_responses[question] = response
    print(f"Q: {question}")
    print(f"A: {response}")
    print("-"*80)

In [ ]:
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

config = SFTConfig(
    output_dir='./output/sft_model',
    save_strategy="epoch",
    save_total_limit=1,
    logging_steps=100,
    max_seq_length=max_length,
    num_train_epochs=3,
    gradient_accumulation_steps=1,
    per_device_train_batch_size=8,
    lr_scheduler_type="constant",
)

trainer = SFTTrainer(
    model,
    args=config,
    train_dataset=ds_train,
    formatting_func=format_input,
    data_collator=collator,
)

### 学習の実行

以下のセルでインストラクションチューニングを実行します。

> **注意**: GPUが必要です。GPUがない環境では学習セル（下記セル）をスキップし、「5.2.3 学習済みモデルによる推論」へ進んでください。

In [ ]:
save_path = "./output/sft_model"
trainer.train()
trainer.save_model(save_path)

## 5.2.3 学習済みモデルによる推論

学習をスキップした場合、または学習済みモデルで推論のみ行いたい場合は、以下のセルを実行してください。ローカルに重みがあればそれを使用し、なければ Hugging Face からダウンロードします。

In [ ]:
import os

# モデルパスの設定
local_model_path = "./output/sft_model"
hf_repo = "elith/llm-book-models"
hf_subfolder = "chapter05/sft_model"

# ローカルに重みがあればそれを使用、なければHFからダウンロード
if os.path.exists(os.path.join(local_model_path, "config.json")):
    print(f"ローカルのモデルを使用: {local_model_path}")
    sft_model = AutoModelForCausalLM.from_pretrained(local_model_path)
    sft_tokenizer = AutoTokenizer.from_pretrained(local_model_path)
else:
    print(f"Hugging Faceからモデルをダウンロード: {hf_repo}/{hf_subfolder}")
    sft_model = AutoModelForCausalLM.from_pretrained(hf_repo, subfolder=hf_subfolder)
    sft_tokenizer = AutoTokenizer.from_pretrained(hf_repo, subfolder=hf_subfolder)

sft_tokenizer.pad_token = sft_tokenizer.eos_token

# デバイス設定
device = "cuda" if torch.cuda.is_available() else "cpu"
sft_model = sft_model.to(device)
print(f"デバイス: {device}")

In [ ]:
# SFTモデルによる推論
test_questions = [
    "What is the capital of Japan?"
    ]

print("="*80)
print("SFTモデルによる推論")
print("="*80)

for question in test_questions:
    response = inference(sft_model, sft_tokenizer, question)
    print(f"\nQ: {question}")
    print(f"A: {response}")
    print("-"*80)